In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile as tf  # write tiff files
from cytocherrypick.calculations import find_median
from PIL import Image  # read tiff files
from toml import load
from tqdm import tqdm  # progress bar

In [2]:
CELL_TYPE = "PBMC"

In [3]:
path = pathlib.Path(f"../results/{CELL_TYPE}/single_cell_predictions.parquet").resolve()

# read in the data
df = pd.read_parquet(path)
# show all columns
pd.set_option("display.max_columns", None)
df.head()

,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,Metadata_Nuclei_Location_Center_Y,Metadata_Nuclei_Location_Center_X,Metadata_Image_Metadata_Site,group,Metadata_Site,Cells_RadialDistribution_ZernikeMagnitude_CorrPM_6_4,Cytoplasm_RadialDistribution_ZernikeMagnitude_CorrER_9_9,Nuclei_RadialDistribution_ZernikePhase_CorrDNA_7_1,Metadata_Well,Metadata_ImageNumber,Metadata_Cells_Number_Object_Number,comparison,treatment,feature,image_compsite_path,image_composite_crop_path,image_DAPI_path,image_ER_path,image_GasderminD_path,image_AGP_path,image_Mitochondria_path,image_compsite_crop_path,image_DAPI_crop_path,image_ER_crop_path,image_GasderminD_crop_path,image_AGP_crop_path,image_Mitochondria_crop_path
0,Flagellin_0.100_ug_per_ml_DMSO_0.025_%,144.003390,424.491525,9,Control,9,1.151881,-0.039144,1.671473,L05,57,248,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
1,DMSO_0.100_%_DMSO_0.025_%,760.351675,1375.437799,9,Control,9,0.834995,-0.402880,1.671473,B07,89,1679,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
2,DMSO_0.100_%_Z-VAD-FMK_100.000_uM,985.997753,242.062921,8,Control,8,-0.758292,-0.128377,1.671471,K12,344,1596,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
3,Disulfiram_1.000_uM_DMSO_0.025_%,2066.174242,134.219697,14,Control,14,4.891341,0.516656,-1.701545,F05,62,4355,control_pyroptosis,control,Nuclei_RadialDistribution_ZernikePhase_CorrDNA...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...,

In [4]:
df.columns
image_path_columns = [
    "image_compsite_path",
    "image_DAPI_path",
    "image_ER_path",
    "image_GasderminD_path",
    "image_AGP_path",
    "image_Mitochondria_path",
    "image_composite_crop_path",
    "image_DAPI_crop_path",
    "image_ER_crop_path",
    "image_GasderminD_crop_path",
    "image_AGP_crop_path",
    "image_Mitochondria_crop_path",
]
for path_type in image_path_columns:
    print(path_type)
    for image_path in df[path_type]:
        print(image_path)
        # open the image
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        # image to array
        image_array = np.array(image)
        resolution = 3.3446  # pixels per micrometer
        # get the image size
        image_size = image_array.shape
        if image_array.shape[1] <= 100:
            scale_bar_length = 5  # um
            scale_bar_height = 1  # pixels
            padding = 3  # pixels
        elif image_array.shape[1] > 100:
            scale_bar_length = 100
            scale_bar_height = 10
            padding = 10
        # get the bottom right most corner based on scale 1 % of pixels
        scale_bar_x = (
            image_size[1] - (scale_bar_length * resolution) - padding - padding
        )
        scale_bar_y = image_size[0] - (scale_bar_height) - padding
        print(scale_bar_x, scale_bar_y)
        # draw the scale bar
        new = cv2.rectangle(
            image_array,
            (int(scale_bar_x), int(scale_bar_y)),
            (image_size[1] - padding, image_size[0] - padding),
            (255, 255, 255),
            -1,
        )
        # show the image
        # plt.imshow(new)
        # plt.show()
        # save the image with the scale bar via cv2
        cv2.imwrite(image_path, new)
#

image_compsite_path
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrDNA_7_1_im1_im3_im4_composite_image_cell_0.png
1805.54 2140
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrDNA_7_1_im1_im3_im4_composite_image_cell_1.png
1805.54 2140
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrDNA_7_1_im1_im3_im4_composite_image_cell_2.png
1805.54 2140
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/figures/3.figure3/figures/composite_images/PBMC/control__control_pyroptosis__Nuclei_RadialDistribution_ZernikePhase_CorrDNA_7_1_im1_im3_im4_composite_image_cell_3.png
1805.54 2140
/home/lippincm/Documents/4TB/data/In